# Machine Learning to Identify Forams species and and whether the shell is cracked or not

## Finnegan Lab
## Data preprocessing

### Setup

In [ ]:
# Imports. Feel free to add or remove as necessary 

# !pip install -Uq tensorflow-datasets==4.5.2
# !pip install tf-nightly -q
import pandas as pd
import numpy as np
np.random.seed(2022)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os, re, math
import keras
import shutil
from pathlib import Path
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#@title mount cloud
from google.colab import auth
auth.authenticate_user()
project_id = 'lucid-loader-347021'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://paleo-ml/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dir_drive_code ='/content/drive/MyDrive/MV1012_SBB_images/ML_projects/Code'
assert os.path.isdir(dir_drive_code), 'unable to find code. Please check your drive path above.'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/BryanZWu/finnegan-paleo-ml.git

In [ ]:
# @title Data stored in drive
dir_base = '/content/drive/MyDrive/MV1012_SBB_images/'
dir_code = '/content/finnegan-paleo-ml/Code'
dir_git_home = '/content/finnegan-paleo-ml'
dir_raw_training_images = f'{dir_base}/Box_Core_images'
dir_ml_home = f'{dir_base}/ML_projects'
dir_google_cloud = 'gs://paleo-ml'
dir_processed_data = dir_out_data = dir_dataset_specs = f'{dir_ml_home}/Processed_data/forams'
dir_training_logs = f'{dir_google_cloud}/Training_logs/'
dir_trained_models = f'{dir_ml_home}/Trained_models/'
dir_train_data = f'{dir_processed_data}/train'
dir_val_data = f'{dir_processed_data}/val'
dir_test_data = f'{dir_processed_data}/test'
dir_debug_processed_data = '/content/training_data_debug'
dir_labels_csv = f'{dir_base}/Final_Diversity_CSVs/ML_master2.csv'

In [ ]:
os.chdir(dir_code)
#@title import local project files
from common.constants import *
from common.imports import *
from data_processing import data_processor
os.chdir(dir_git_home)

In [ ]:
def git():
    """ Refetch all files """
    !git fetch
    !git checkout origin -- .

## Processing data and storing

Take data from its original form and crop/resize images as necessary. Store images with its corresponding label.

### Run processing

In [ ]:
labels_df = data_processor.create_labels_df(dir_labels_csv)
labels_df.head()


Broken            species     age   test    val
sample_name  object_num                                                   
MV1012-BC-12 1.0         unbroken          NOT FORAM  1994.0  False  False
             2.0         unbroken   suggrunda eckisi  1994.0  False  False
             3.0           broken    bulimina exilis  1994.0  False  False
             4.0         unbroken  nonionella stella  1994.0  False  False
             5.0         unbroken          NOT FORAM  1994.0  False   True

In [ ]:
#Run data processing
data_processor.run_processing(dir_debug_processed_data, dir_raw_training_images, labels_df)

## Create tensorflow dataset and push to google cloud

In [ ]:
git()

In [ ]:
data_processor.create_cloud_dataset(dir_debug_processed_data, dir_google_cloud, dir_dataset_specs, 256)

<PrefetchDataset element_spec={'chamber_broken': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image': TensorSpec(shape=(416, 416, 3), dtype=tf.uint8, name=None), 'species': TensorSpec(shape=(), dtype=tf.int64, name=None)}>
